In [1]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2015' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2015.csv
Star Wars: Episode VII - The Force Awakens 2015-12-18
Jurassic World 2015-06-12
Furious 7 2015-04-03
Avengers: Age of Ultron 2015-05-01
Minions 2015-07-10
Spectre 2015-11-06
Inside Out 2015-06-19
Mission: Impossible - Rogue Nation 2015-07-31
The Hunger Games: Mockingjay - Part 2 2015-11-20
The Martian 2015-10-02
Fifty Shades of Grey 2015-02-13
Cinderella 2015-03-13
The Revenant 2016-12-25
Ant-Man 2015-07-17
Hotel Transylvania 2 2015-09-25
San Andreas 2015-05-29
Terminator Genisys 2015-07-01
Kingsman: The Secret Service 2015-02-13
Home 2015-03-27
Mad Max: Fury Road 2015-05-15
The Good Dinosaur 2015-11-25
Taken 3 2015-01-09
The SpongeBob Movie: Sponge Out of Water 2015-02-06
Maze Runner: The Scorch Trials 2015-09-18
The Divergent Series: Insurgent 2015-03-20
Pitch Perfect 2 2015-05-15
The Peanuts Movie 2015-11-06
Pixels 2015-07-24
Daddy's Home 2015-12-25
Spy 2015-06-05
Alvin and the Chipmunks: The Road Chip 2015-12-18
Ted 2 2015-06-26
Tomorrowland 2015-05-22
Ever